Step 1: Setup OpenAI API Key

In [ ]:
import openai
from openai import OpenAI

api_key="<your api-key>"


Step 2: Reading training file with 11 examples

In [4]:
data_file = "Assignment/data/emoji_test_train.jsonl"
# Simple text file reading
with open(data_file, "r") as file:
    content = file.read()
    print(len(content),'\n',content)

2279 
 {"messages": [{"role": "system", "content": "You're a chatbot that only responds with emojis!"}, {"role": "user", "content": "I just passed my driving test!"}, {"role": "assistant", "content": "(party)" }]}  
{"messages": [{"role": "system", "content": "You're a chatbot that only responds with emojis!"}, {"role": "user", "content": "Feeling pretty sick today."}, {"role": "assistant", "content": "(facewithmedicalmask)" }]}   
{"messages": [{"role": "system", "content": "You're a chatbot that only responds with emojis!"}, {"role": "user", "content": "I just got promoted at work!"}, {"role": "assistant", "content": "(confetti)" }]}  
{"messages": [{"role": "system", "content": "You're a chatbot that only responds with emojis!"}, {"role": "user", "content": "I'm feeling really tired."}, {"role": "assistant", "content": "(sleeping)" }]}  
{"messages": [{"role": "system", "content": "You're a chatbot that only responds with emojis!"}, {"role": "user", "content": "Just adopted a puppy!

Step 3: Upload Training File to OpneAI

In [5]:
import openai
from openai import OpenAI

#Make API Client to interact ChatGPT
client = OpenAI(api_key=api_key)

data_file = "Assignment/data/emoji_test_train.jsonl"

# Upload training file
with open(data_file, "rb") as file:
    uploaded_file = client.files.create(
        file=file,
        purpose="fine-tune"
    )

print("Uploaded file ID:", uploaded_file.id)


Uploaded file ID: file-Bemda6VmBac3dqSojhfSJQ


Step 4: Create and Start a Fine-Tunning Job

In [6]:
# Start a fine-tuning job
fine_tuning_job = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    model="gpt-3.5-turbo",  # Base model to fine-tune
    suffix="emoji-bot"  # Custom suffix for your model name
)

# Save the job ID
job_id = fine_tuning_job.id
print(f"Fine-tuning job created with ID: {job_id}")

Fine-tuning job created with ID: ftjob-ErAzyriYoGsYjeVc5gqLQ7RD


Step 5: Monitor the Fine-Tuning Progress

In [12]:
# Check the status of your fine-tuning job
job_status = client.fine_tuning.jobs.retrieve(job_id)
print(f"Job status: {job_status.status}")

# List events to see progress details
events = client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=job_id, 
    limit=10
    )

for event in events.data:
    print(f"Time({event.created_at}): {event.message}")

Job status: succeeded
Time(1745767104): The job has successfully completed
Time(1745767099): New fine-tuned model created
Time(1745767099): Checkpoint created at step 88
Time(1745767099): Checkpoint created at step 77
Time(1745767063): Step 99/99: training loss=0.00
Time(1745767063): Step 98/99: training loss=0.00
Time(1745767061): Step 97/99: training loss=0.00
Time(1745767060): Step 96/99: training loss=0.00
Time(1745767057): Step 95/99: training loss=0.86
Time(1745767057): Step 94/99: training loss=0.00


Step 6: Crate Fine-Tuned Model Endpoint

In [13]:
# Get the model endpoint
completed_job = client.fine_tuning.jobs.retrieve(job_id)
model_endpoint = completed_job.fine_tuned_model
print(f"Your fine-tuned model endpoint: {model_endpoint}")

Your fine-tuned model endpoint: ft:gpt-3.5-turbo-0125:personal:emoji-bot:BQy3mqFx


Step 7: Use Fine-Tuned Model Endpoint to Validate four different emotions

In [24]:
# Test your fine-tuned model
user_content = ['I scrwed my lunch today!','I just landed in India!','I lost my best friend.','That joke was so funny!','My son statretd to walk today.']

for user in user_content:
    response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:emoji-bot:BQynTGb1", #Model took long time to get ready, so I updated the endpoint
    messages=[
        {"role": "system", "content": "You are an Emoji Bot that converts text descriptions to appropriate emojis."},
        {"role": "user", "content": user}
    ],
    max_tokens=50
    )
    print('User Emotion: ',user,'\n Emoji Bot Representation: ',response.choices[0].message.content.strip('()'))

User Emotion:  I scrwed my lunch today! 
 Emoji Bot Representation:  poop
User Emotion:  I just landed in India! 
 Emoji Bot Representation:  india
User Emotion:  I lost my best friend. 
 Emoji Bot Representation:  sadcat
User Emotion:  That joke was so funny! 
 Emoji Bot Representation:  rofl
User Emotion:  My son statretd to walk today. 
 Emoji Bot Representation:  child


Step 8: Crate FineTune model with another training file with 569 examples, last one had 11 examples

In [25]:
import openai
from openai import OpenAI

data_file = "Assignment/data/emoji_test_more_train.jsonl"

# Upload training file with more data
with open(data_file, "rb") as file:
    uploaded_file = client.files.create(
        file=file,
        purpose="fine-tune"
    )

print("Uploaded file ID:", uploaded_file.id)

# Start a fine-tuning job
fine_tuning_job = client.fine_tuning.jobs.create(
    training_file=uploaded_file.id,
    model="gpt-3.5-turbo",  # Base model to fine-tune
    suffix="emoji-bot"  # Custom suffix for your model name
)

# Save the job ID
job_id = fine_tuning_job.id
print(f"Fine-tuning job created with ID: {job_id}")

# Check the status of your fine-tuning job
job_status = client.fine_tuning.jobs.retrieve(job_id)
print(f"Job status: {job_status.status}")

# List events to see progress details
events = client.fine_tuning.jobs.list_events(
    fine_tuning_job_id=job_id, 
    limit=10
    )

for event in events.data:
    print(f"Time({event.created_at}): {event.message}")
    
#Use Fine-Tuned Model Endpoint to Validate same three different emotions
# Test your fine-tuned model
user_content = ['I scrwed my lunch today!','I just landed in India!','I lost my best friend.','That joke was so funny!','My son statretd to walk today.']

print('\n\t ***** Model Fine Tune with more data ******\n\n')

for user in user_content:
    response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:emoji-bot:BQywx1HQ", #model took time to get ready, so I updated the endpoint
    messages=[
        {"role": "system", "content": "You are an Emoji Bot that converts text descriptions to appropriate emojis."},
        {"role": "user", "content": user}
    ],
    max_tokens=50
    )
    print('User Emotion: ',user,'\n Emoji Bot Representation: ',response.choices[0].message.content.strip('()'))

Uploaded file ID: file-SfXX9MY2iXZV2eijaWa1VX
Fine-tuning job created with ID: ftjob-r9T7PShcc60Z4SjmxahEy7R2
Job status: validating_files
Time(1745771510): Validating training file: file-SfXX9MY2iXZV2eijaWa1VX
Time(1745771510): Created fine-tuning job: ftjob-r9T7PShcc60Z4SjmxahEy7R2

	 ***** Model Fine Tune with more data ******


User Emotion:  I scrwed my lunch today! 
 Emoji Bot Representation:  poop
User Emotion:  I just landed in India! 
 Emoji Bot Representation:  airplane
User Emotion:  I lost my best friend. 
 Emoji Bot Representation:  sadcat
User Emotion:  That joke was so funny! 
 Emoji Bot Representation:  rofl
User Emotion:  My son statretd to walk today. 
 Emoji Bot Representation:  babylearnstowalk


Here, the model with more training data gave better responses. While we updated the bot about India landing, it gave an airplane response rather giving irrelevant response stating the country. 
The same goes on for our last query, the response is more relevent to the query than the previous one trained with less examples.


Step 9: Delete fineTune endpoint to save cost (if not needed) 

In [26]:
# #Created Models
model_list=["ft:gpt-3.5-turbo-0125:personal:emoji-bot:BQyaylS7"]

for list in model_list:
    client.models.delete(list)